#### 1. Bibliothèques

In [7]:
pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### 2. Chargement des donées


In [7]:
import pandas as pd
file1='Data clear/fichier_combiné.csv'
df = pd.read_csv(file1)
print(df.shape,df.head())

(33471, 2)                                                Block  Label
0  ﻿The Project Gutenberg eBook of Contes Françai...      3
1  You may copy it, give it away or re-use it und...      3
2  If you are not located in the United States, y...      3
3  Title: Contes Français Editor: Douglas Labaree...      3
4                                                 D.      3


In [29]:
# Mélanager le dataframe
df_suffle = df.sample(frac=0.05, random_state=42).reset_index(drop=True)
print(df_suffle.shape)
print(df_suffle.head())

(1674, 2)
                                               Block  Label
0  Un cordon de grosses pierres fixait tout autou...      1
1  Vous dirai-je pourtant que j'ai perdu quelques...      2
2  lèvre, _f._ lip; lèvres en fleur, full-blown l...      3
3  Bonaparte (Napoléon), the French Emperor (1769...      3
4  mengheldichten: fyghes noeper;  Bacchus-Cortryck.      2


In [30]:
texts = df_suffle['Block'].tolist()
labels = df_suffle['Label'].tolist()

In [31]:
# Prétraiter les données
from datasets import Dataset

dataset = Dataset.from_dict({'text': texts, 'label': labels})
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1674 [00:00<?, ? examples/s]

#### Division des donnés

In [32]:
from sklearn.model_selection import train_test_split


In [33]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.5, random_state=42)


In [34]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

In [35]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 837
})
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 837
})


#### 5. Chargement du model

In [36]:
from transformers import CamembertForSequenceClassification


In [37]:
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(set(labels)))


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 6. Entrainement du model

In [38]:
from transformers import Trainer, TrainingArguments

In [39]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

/home/yop/project_dumas2/.venv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/yop/project_dumas2/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate()
print(results)

#### 8. Prédiction sur de nouvelles données


In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = logits.argmax().item()
    return prediction

In [ ]:
# Exemple de prédiction
new_text = "Votre texte à classer ici"
predicted_label = predict(new_text)
print(predicted_label)

#### 9. Sauvegarde et chargement du model 

In [ ]:
model.save_pretrained('./camembert_classification_model')
tokenizer.save_pretrained('./camembert_classification_model')

# Pour charger
model = CamembertForSequenceClassification.from_pretrained('./camembert_classification_model')
tokenizer = CamembertTokenizer.from_pretrained('./camembert_classification_model')